# Data extraction and transformation
- GET the data from twitter's api
- Transform the data to find what we want

In [ ]:
import requests
from requests_oauthlib import OAuth1
import pprint
import json
from pandas.io.json import json_normalize
import time
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
###obtain consumer key/secret and token key/secret from twitter dev site.
#consumer key/secret
c_key = ""
c_secret = ""

#token key/secret
key = ""
secret = ""

# for printing json and objects in easy-to-read format
pp = pprint.PrettyPrinter(indent=1)

In [ ]:
# Define the search url
searchURL_init = 'https://api.twitter.com/1.1/search/tweets.json'
auth = OAuth1(c_key, c_secret,key, secret)

In [ ]:
# helper function that takes the JSON data and transforms it into dataframes
def append_dataframes(result, data): #[tweetsDf, usersDf]
    r = result['statuses']
    for i in range(len(r)):
        #populate the tweetsDf
        userId = r[i]['user']['id_str']
        created_at = r[i]['created_at']
        tweet_text = r[i]['text']
        tweetId = r[i]['id_str']
        data[0] = data[0].append(pd.DataFrame(data={'authorId':userId, 'created_at':created_at, 'tweet_text':tweet_text},
                                              index=[tweetId]))

        #populate the usersDf
        if userId not in data[1].index:
            name = r[i]['user']['name']
            screen_name = r[i]['user']['screen_name']
            data[1] = data[1].append(pd.DataFrame(data={'name':name, 'screen_name':screen_name}, index=[userId]))
            
        #populate the userMentionsDf
        all_mentions = r[i]['entities']['user_mentions']
        for user in range(len(all_mentions)):
            userId_mention = all_mentions[user]['id_str']
            name_mention = [all_mentions[user]['name']]
            screenName_mention = [all_mentions[user]['screen_name']]
            data[2] = data[2].append(pd.DataFrame({'mentions_userId':[userId_mention], 'tweetId':[tweetId]}),
                                                   ignore_index=True)
            if userId_mention not in data[1].index:
                data[1] = data[1].append(pd.DataFrame(data={'name':name_mention, 'screen_name':screenName_mention}, 
                                                  index=[userId_mention]))

    return data


In [ ]:
# helper function that calls the twitter API and handles rate limiting
def api_call(dataframes=None, tweetCount=5, raw=True): # dataframes as list
    if tweetCount%100.0 == 0:
        total_pages = tweetCount/100
    else:
        total_pages = (tweetCount/100) + 1
    if tweetCount<=100:
        count = tweetCount
    else:
        count = 100
    params = {"q":"@dominos #easyorder","count":count, 'result_type':'recent'} #initial parameters
    next_results = ""
    call_log = []
    page = 1
    for i in range(total_pages):
        searchURL = searchURL_init + next_results
        response = requests.get(searchURL, auth=auth, params=params)
        call_log.append(response.status_code)
        result = response.json()
        try:
            next_results = result['search_metadata']['next_results'] #use twitter's cursor to get the next page of results
        except KeyError:
            if page != total_pages:
                print 'API limit reached... snoozing 16 minutes'
                time.sleep(360)
                print 'API limit reached... 10 minutes remaining'
                time.sleep(300)
                print 'API limit reached... 5 minutes remaining'
                time.sleep(240)
                print 'API limit reached... 1 minutes remaining'
                time.sleep(60)
                print 'Rockets refueled... here we go.'
                continue
        if not raw:
            dfs = append_dataframes(result, dataframes)
        params = None
        page += 1
        
    if not raw:
        return dfs, call_log
    else:
        return result, call_log

## Exploration
What does the dataset look like?

In [ ]:
test, log = api_call(tweetCount=1)
print test.keys()

In [ ]:
#explore the metadata
pp.pprint(test['search_metadata'])

In [ ]:
#explore the statuses
pp.pprint(test['statuses'][0])

In [ ]:
# find structure of retweets
for status in range(len(test['statuses'])):
    amt = test['statuses'][status]['entities']['user_mentions']
    if len(amt)>2:
        pp.pprint(test['statuses'][status])

In [ ]:
#create emtpy dataframes
tweetsDf = pd.DataFrame(columns=['authorId', 'created_at', 'tweet_text']) 
usersDf = pd.DataFrame(columns=['name', 'screen_name'])
userMentionsDf = pd.DataFrame(columns=['mentions_userId', 'tweetId'])
dfList = [tweetsDf, usersDf, userMentionsDf]

#Call the api and append data to empty dataframes
dataframes, log = api_call(dataframes=dfList, tweetCount=350, raw=False)
dataframes[0].index.name = 'tweetId'
dataframes[1].index.name = 'userId'
print 'tweetsDf: {}\n{}\n\n'.format(dataframes[0].shape, dataframes[0].head())
print 'usersDf: {}\n{}\n\n'.format(dataframes[1].shape, dataframes[1].head())
print 'userMentionsDf: {}\n{}\n\n'.format(dataframes[2].shape, dataframes[2].head())

In [ ]:
#Remove dataframes from the list
tweetsDf = dfList[0]
usersDf = dfList[1]
userMentionsDf = dfList[2]

# find unique tweets to check the api did not duplicate any
dupTest = tweetsDf.groupby(tweetsDf.index).first().shape
print dupTest
print 'There are duplicate tweetIds:', dupTest[0]!=tweetsDf.shape[0]
print tweetsDf[tweetsDf.index.duplicated()]

In [ ]:
##### mentions dataframe #####
# for each tweet show all user mentions
data = pd.merge(userMentionsDf, tweetsDf, left_on='tweetId', right_index=True) 

##### welcome dataframe #####
# create welcome dataframe: shows all tweets that welcome a new tweet order signup
welcomeDf = data[(data.authorId == '31444922') & (data.mentions_userId != '31444922')]
welcomeDf = welcomeDf.rename(columns={'created_at':'welcome_datetime'})

# subset for only relevant columns
welcomeDf = welcomeDf[['mentions_userId', 'welcome_datetime']]
print 'welcomeDf: {}\n{}\n\n'.format(welcomeDf.shape, welcomeDf.head())


##### orders dataframe #####
# create orders dataframe: show all tweets that ordered a pizza
ordersDf = data[(data.authorId != '31444922') & (data.mentions_userId == '31444922')]
ordersDf = ordersDf.rename(columns={'created_at':'order_datetime'})
ordersDf = ordersDf.drop_duplicates() # for retweets @dominos is mentions 2x and the author 1x, this removes the dups

# subset for only the relevant columns and groupby to find the min(orderDate), aka: first order
ordersDf = ordersDf[['authorId', 'order_datetime']]
ordersDf = ordersDf.groupby('authorId', as_index=False).min()
print 'ordersDf: {}\n{}\n\n'.format(ordersDf.shape, ordersDf.head())

# create duration dataframe: show the difference in activation time and order time
durationDf = pd.merge(welcomeDf, ordersDf, left_on='mentions_userId', right_on='authorId')
durationDf['duration_seconds'] = np.NaN

for index, row in durationDf.iterrows():
    orderDate = datetime.strptime(row[3].replace('+0000 ',''), "%a %b %d %H:%M:%S %Y")
    welcomeDate = datetime.strptime(row[1].replace('+0000 ',''), "%a %b %d %H:%M:%S %Y")
    durationDf.loc[index, 'order_datetime'] = orderDate
    durationDf.loc[index, 'welcome_datetime'] = welcomeDate
    diff = orderDate - welcomeDate
    durationDf.loc[index, 'duration_seconds'] = diff.total_seconds()

print 'durationDf: {}\n{}\n\n'.format(durationDf.shape, durationDf.head(5))

In [ ]:
# min welcome and order dates
welcome_min = durationDf.welcome_datetime.min()
order_min = durationDf.order_datetime.min()
welcome_max = durationDf.welcome_datetime.max()
order_max = durationDf.order_datetime.max()

print 'min date:', np.min([welcome_min, order_min])
print 'max date:', np.max([welcome_max, order_max])
print '\n'
print 'tweet date range:', np.max([welcome_max, order_max]) - np.min([welcome_min, order_min])

In [ ]:
# export the resulting dataframes (Windows)
directoryLoc = ""
welcomeDf.to_csv(directoryLoc + "\welcomeDf.csv", index=False)
ordersDf.to_csv(directoryLoc + "\ordersDf.csv", index=False)
durationDf.to_csv(directoryLoc + "\durationDf.csv", index=False)

# ANALYSIS


## Contents
### 1. Dataset structure
- What does the dataset look like?

### 2. Results (questions)
1. How many customers actually place an order within 6 days after signing up for Tweet Ordering?
2. What is the median time difference between when a customer signs up Tweet Ordering and when they place their first order with Tweet Ordering? 

# Results
### Conversion rate: who actually ordered?

In [ ]:
print 'Total new Tweet Ordering signups: {}'.format(len(welcomeDf))
print 'Total first time orders (only those from the previous total): {}'.format(len(durationDf))
conversion = round((len(durationDf)/float(len(welcomeDf))*100),2)
print 'Conversion rate: {}%\n'.format(conversion)
print '##### CONCLUSION #####'
print 'About {}% of customers that signed up for Tweet Ordering in the past 6 days placed an order with the same time period.'\
        .format(conversion)

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        return '{p:.2f}%  ({v:d})'.format(p=pct,v=val)
    return my_autopct

labels = 'Activations', 'Placed first order'
values = [len(welcomeDf), float(len(durationDf))]
explode = (0, 0.35)  # display the orders prominently
plt.subplots()
patches, texts, autotexts = plt.pie(sizes, explode=explode, labels=labels, autopct=make_autopct(values), startangle=0)
plt.axis('equal', pickradius=30)  # Equal aspect ratio ensures that pie is drawn as a circle.


plt.show()